In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
# deprecated/no longer used: client.list_experiments()
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp-mausul/02-experiment-tracking/mlruns/1', creation_time=1716274678962, experiment_id='1', last_update_time=1716274678962, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716272965747, experiment_id='0', last_update_time=1716272965747, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
client.create_experiment(name='cool-experiment')

'2'

In [22]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.rmse < 6.33',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [23]:
for run in runs:
    print(f"Run ID: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

Run ID: 15b97c40d9b1416e8168aab8c633f166, rmse: 6.3111
Run ID: d98e2f264771442b9319eecfc6e4c188, rmse: 6.3111
Run ID: 285b0abeb7f54db985750697b3189326, rmse: 6.3111
Run ID: 5c75b5479a3f409281d26010655ef050, rmse: 6.3111
Run ID: 1f2707e2a639471e9181e192390bc56e, rmse: 6.3205


In [21]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [26]:
run_id = "1f2707e2a639471e9181e192390bc56e"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716531536465, current_stage='None', description=None, last_updated_timestamp=1716531536465, name='nyc-taxi-regressor', run_id='1f2707e2a639471e9181e192390bc56e', run_link=None, source='/workspaces/mlops-zoomcamp-mausul/02-experiment-tracking/mlruns/1/1f2707e2a639471e9181e192390bc56e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [29]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716526290025, description='These models are used for estimating trip duration in New York City.\n', last_updated_timestamp=1716531536465, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716531536465, current_stage='None', description=None, last_updated_timestamp=1716531536465, name='nyc-taxi-regressor', run_id='1f2707e2a639471e9181e192390bc56e', run_link=None, source='/workspaces/mlops-zoomcamp-mausul/02-experiment-tracking/mlruns/1/1f2707e2a639471e9181e192390bc56e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>], name='nyc-taxi-regressor', tags={}>]

In [31]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_5107/3294275207.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [32]:
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1716531536465, current_stage='None', description=None, last_updated_timestamp=1716531536465, name='nyc-taxi-regressor', run_id='1f2707e2a639471e9181e192390bc56e', run_link=None, source='/workspaces/mlops-zoomcamp-mausul/02-experiment-tracking/mlruns/1/1f2707e2a639471e9181e192390bc56e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>]

In [43]:
for version in latest_versions:
    print(f"version: {version.version}, state: {version.status}")

version: 4, state: READY


In [37]:
# client.transition_model_version_stage(name, version, stage) is deprecated
client.set_registered_model_alias(name=model_name, alias='not-best-but-ok', version=4)

In [39]:
client.update_model_version(name=model_name, version=4, description='Alias is used instead of staging for model registry')

<ModelVersion: aliases=['not-best-but-ok'], creation_timestamp=1716531536465, current_stage='None', description='Alias is used instead of staging for model registry', last_updated_timestamp=1716532338103, name='nyc-taxi-regressor', run_id='1f2707e2a639471e9181e192390bc56e', run_link=None, source='/workspaces/mlops-zoomcamp-mausul/02-experiment-tracking/mlruns/1/1f2707e2a639471e9181e192390bc56e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [81]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, version, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{version}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [47]:
!ls

data				       mlflow.db
duration-prediction.ipynb	       mlruns
exp_tracking_and_model_registry.ipynb  models
mlflow-exploration.ipynb	       requirements.txt


In [50]:
df = read_dataframe("./data/green_tripdata_2021-03.parquet")

In [51]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.5,...,0.5,0.00,0.0,None,0.3,8.80,1.0,1.0,0.0,8.350000
1,2,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.0,...,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.0,5.233333
2,2,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.0,...,0.5,2.00,0.0,None,0.3,31.30,1.0,1.0,0.0,20.333333
3,2,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.5,...,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.0,7.666667
4,1,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.0,...,0.5,1.85,0.0,None,0.3,8.15,1.0,1.0,0.0,3.600000


In [53]:
run_id = "15b97c40d9b1416e8168aab8c633f166"

In [54]:
# download dict vectorizer from mlflow (previously stored)
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp-mausul/02-experiment-tracking/preprocessor'

In [60]:
!ls

data				       mlruns
duration-prediction.ipynb	       models
exp_tracking_and_model_registry.ipynb  preprocessor
mlflow-exploration.ipynb	       requirements.txt
mlflow.db


In [61]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [63]:
X_test = preprocess(df, dv)

In [64]:
target = "duration"
y_test = df[target].values

In [66]:
model_name

'nyc-taxi-regressor'

In [86]:
version=1

In [84]:
# FAQ/Slack: MlflowException: Not a proper models:/ URI: models:/nyc-taxi-regressor. Models URIs must be of the form 'models:/model_name/suffix' or 'models:/model_name@alias' where suffix is a model version, stage, or the string 'latest' and where alias is a registered model alias. Only one of suffix or alias can be defined at a time.

In [83]:
%time test_model(model_name, version, X_test, y_test)

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:29:50] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 30.6 s, sys: 37 ms, total: 30.6 s
Wall time: 16.9 s


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 6.258292703800276}